# Heston Nandi Options

In [16]:
%load_ext autoreload
%autoreload 2
import finoptions as fo
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
lamb=-0.5; omega=2.3e-6; alpha=2.9e-6; beta=0.85; gamma=184.25; S=100; K=100; t=252; r=0.05/252